In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from source.datasets.fast_datasets import *


tasks_db = [
    {
        "task_name": "GTZAN",
        "dataset": GTZANFastDataset,
        "color": "red",
        "epochs": 1,
        "workers": 2,
    },
    {
        "task_name": "EmoMusic",
        "dataset": EmoMusicFastDataset,
        "color": "blue",
        "epochs": 1,
        "workers": 2,
    },
    {
        "task_name": "Deezer",
        "dataset": DeezerFastDataset,
        "color": "green",
        "epochs": 1,
        "split_size": 0.9,
        "workers": 2,
    },
    {
        "task_name": "MagnaTagATune",
        "dataset": MTATFastDataset,
        "color": "yellow",
        "epochs": 1,
        "split_size": 0.9,
        "workers": 2,
    },
]

In [3]:
from source.evaluation import fine_tune_model



#model = "models/music_bert_mlm_ce.pth"
model = None
losses, vals = [], []

executor = ProcessPoolExecutor() # ThreadPoolExecutor() #

pool = []

for item in tasks_db[:2]:
    dataset = item["dataset"]
    epochs = item["epochs"]
    
    kwargs = {'name':item['task_name'],
              'dataset_class':dataset,
              'epochs':epochs,
              'evals':10/epochs,
              'load_model_path':model,
              'split_size':item.get("split_size"),
              'workers': item.get("workers")
             }
    
    pr = Process(fine_tune_model, **kwargs)
    thread = executor.submit(fine_tune_model, **kwargs)
    pool.append(thread)

    
#     loss, val = fine_tune_model(dataset=dataset,
#                                 epochs=epochs,
#                                 evals=10/epochs,
#                                 load_model_path=model,
#                                 split_size=item.get("split_size"),
#                                 workers=item.get("workers"))
    
for thread in pool:
    try:
        loss, val = thread.result()
    except Exception as e:
        print(e)
        raise e

    losses.append(loss)
    vals.append(val)

Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method


RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

In [3]:
%matplotlib widget
from source.utils.plot_utils import plot_curve, smooth

for item, val in zip(tasks_db, vals):
    plot_curve(val, 1, color = item["color"], label=item["task_name"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
%matplotlib widget

for item, loss in zip(tasks_db, losses):
    loss = smooth(loss, 2)

    plot_curve(loss, 1, color = item["color"], label=item["task_name"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
from source.utils.generic_utils import allDone

allDone()